In [26]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
import numpy as np


In [7]:
messages = pd.read_csv('spam.csv',encoding='latin-1')

messages.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
messages.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)
messages.rename(columns={
    'v1' : 'label',
    'v2' : 'message'
},inplace=True)

In [9]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
lemmatizer = WordNetLemmatizer()

In [74]:
import re

corpus = []
for i in range(0,len(messages)) :
    review = str(messages['message'][i]) 
    review= re.sub('[^a-zA-Z]',' ',review)
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [76]:
len(messages)

5572

In [77]:
len(corpus)

5572

In [81]:
from gensim.utils import simple_preprocess

In [79]:
words = []

for sent in corpus :
    sent_token = simple_preprocess(sent)
    words.append(sent_token)

In [ ]:
model = gensim.models.Word2Vec(words,vector_size=100,)


In [104]:
model.wv.index_to_key

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'ok',
 'day',
 'free',
 'know',
 'come',
 'like',
 'time',
 'good',
 'got',
 'text',
 'love',
 'want',
 'send',
 'one',
 'need',
 'txt',
 'today',
 'going',
 'home',
 'stop',
 'lor',
 'sorry',
 'see',
 'mobile',
 'still',
 'take',
 'back',
 'da',
 'reply',
 'think',
 'tell',
 'dont',
 'week',
 'phone',
 'hi',
 'new',
 'later',
 'pls',
 'please',
 'co',
 'msg',
 'min',
 'dear',
 'make',
 'night',
 'message',
 'well',
 'say',
 'thing',
 'much',
 'claim',
 'great',
 'hope',
 'oh',
 'hey',
 'number',
 'happy',
 'friend',
 'wat',
 'give',
 'work',
 'way',
 'yes',
 'www',
 'let',
 'prize',
 'right',
 'tomorrow',
 'already',
 'tone',
 'ask',
 'said',
 'cash',
 'win',
 'yeah',
 'amp',
 'really',
 'life',
 'im',
 'babe',
 'meet',
 'find',
 'morning',
 'last',
 'service',
 'year',
 'uk',
 'thanks',
 'miss',
 'would',
 'com',
 'also',
 'nokia',
 'anything',
 'lol',
 'care',
 'every',
 'feel',
 'keep',
 'pick',
 'sure',
 'sent',
 'contact',
 'urgent',


In [105]:
model.corpus_count

5572

In [106]:
model.epochs

5

In [107]:
model.wv.similar_by_word('boy')

[('one', 0.9995939135551453),
 ('girl', 0.9995764493942261),
 ('find', 0.9995704293251038),
 ('day', 0.9995666146278381),
 ('back', 0.9995563626289368),
 ('even', 0.9995424151420593),
 ('amp', 0.9995168447494507),
 ('would', 0.9995051026344299),
 ('always', 0.9994986057281494),
 ('miss', 0.9994967579841614)]

In [108]:
model.wv.similar_by_word('good')

[('day', 0.9995614290237427),
 ('much', 0.9995419979095459),
 ('dear', 0.9995054006576538),
 ('give', 0.9995037317276001),
 ('go', 0.9995021224021912),
 ('even', 0.999491810798645),
 ('miss', 0.9994913935661316),
 ('want', 0.9994913339614868),
 ('going', 0.9994861483573914),
 ('hope', 0.9994854927062988)]

In [129]:
def avgWord2vec(doc):
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if len(vectors) == 0:
        return np.zeros(100) 
    return np.mean(vectors,axis=0)


In [131]:
from tqdm import tqdm

In [132]:
X = []

for i in tqdm(range(len(words))) :
    X.append(avgWord2vec(words[i]))

100%|██████████| 5572/5572 [00:01<00:00, 5428.88it/s]


In [133]:
# independent feature
X_new = np.array(X)

In [166]:
X_new[0].shape

(100,)

In [165]:
X_new.shape

(5572, 100)

In [167]:
# dependent feature

y = pd.get_dummies(messages['label']).astype(int)
y = y.iloc[:,0].values

In [168]:
y.shape

(5572,)

In [146]:
# final independent feature

df = pd.DataFrame(X_new)


In [147]:
df.shape

(5572, 100)

In [148]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.140449,0.261008,-0.006692,0.020733,0.045894,-0.379237,0.056731,0.458892,-0.104325,-0.114286,...,0.327410,0.161860,-0.044713,0.045380,0.420630,0.301712,0.104361,-0.240211,0.132002,-0.024131
1,-0.116415,0.217234,-0.009158,0.016257,0.035248,-0.318991,0.056520,0.391632,-0.092440,-0.093596,...,0.278865,0.139015,-0.040121,0.036919,0.353456,0.255922,0.088908,-0.208504,0.111951,-0.015906
2,-0.151026,0.280809,-0.011135,0.032715,0.047502,-0.389457,0.052385,0.472720,-0.109304,-0.106014,...,0.337063,0.169026,-0.043893,0.036239,0.433851,0.317344,0.107218,-0.252708,0.140034,-0.015815
3,-0.202277,0.370520,-0.005494,0.032764,0.068637,-0.550728,0.083201,0.665242,-0.148007,-0.163188,...,0.465383,0.231890,-0.061184,0.068659,0.600926,0.434100,0.141880,-0.343191,0.188933,-0.034587
4,-0.160331,0.296218,-0.004646,0.023080,0.048554,-0.431215,0.066508,0.522536,-0.122628,-0.127588,...,0.374940,0.186448,-0.055735,0.049677,0.479574,0.346001,0.120159,-0.276493,0.149760,-0.026784


In [152]:
X = df


In [153]:
X.isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
95    0
96    0
97    0
98    0
99    0
Length: 100, dtype: int64

## Model Training

In [155]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [156]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()

In [161]:
classifier.fit(X_train,y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [163]:
y_pred = classifier.predict(X_test)

In [164]:
from sklearn.metrics import accuracy_score,classification_report

print(f"Accuracy : {accuracy_score(y_test,y_pred)}\nClassification Report :\n{classification_report(y_test,y_pred)}")

Accuracy : 0.9677033492822966
Classification Report :
              precision    recall  f1-score   support

           0       0.93      0.81      0.87       218
           1       0.97      0.99      0.98      1454

    accuracy                           0.97      1672
   macro avg       0.95      0.90      0.92      1672
weighted avg       0.97      0.97      0.97      1672

